# 3.2. Maps and Colors

Load data 

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import json
import urllib.request 

In [ ]:
def layout_helper(fig):
    fig.update_traces(marker_line_width=0.1)
    return fig

In [ ]:
with urllib.request.urlopen(
    "https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/ct_shape_int.geojson") as url:
    cantons = json.loads(url.read().decode())

In [ ]:
with urllib.request.urlopen(
    "https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/gmd_shape_int.geojson") as url:
    communities = json.loads(url.read().decode())

In [ ]:
#communities

In [ ]:
df = pd.read_csv('https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/cantons.csv',dtype={'canton_code':'str'})

In [ ]:
df.head()

In [ ]:
df = df[df['year'] == 2018]

In [ ]:
df_comm = pd.read_csv('https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/communities.csv')
df_comm = df_comm[df_comm['year']==2017]

In [ ]:
df_comm.head(3)

As we will build the same type of visualisation over and over again, let's put this into a function.

In [ ]:
def map_viz(df, geodata, geocode_df, geocode_json, colorvar):
    fig = px.choropleth_mapbox(df, geojson=geodata,
                           locations=df[geocode_df],
                           featureidkey="properties.{}".format(geocode_json),
                           color=colorvar,
                           mapbox_style="white-bg",
                           zoom=6.3, 
                           center = {"lat": 46.8, "lon": 8.5},
                           width=900,height=500
                           )
    layout_helper(fig).show()

## How to discretise?
### 1: Quantile bins

In [ ]:
df_comm['color_bins_n_10'] = pd.qcut(df_comm['taxable_income_per_capita'], q=10,labels=False)
df_comm['color_bins_n_10_description'] = pd.qcut(df_comm['taxable_income_per_capita'], q=10)

In [ ]:
df_comm.head(3)

In [ ]:
df_comm[['color_bins_n_10','color_bins_n_10_description']].drop_duplicates().sort_values(by='color_bins_n_10')

In [ ]:
map_viz(df_comm,communities,'GMDNR','GMDNR','color_bins_n_10')

### 2: Fisher Jenks

In [ ]:
import jenkspy

In [ ]:
breaks = jenkspy.jenks_breaks(df_comm['taxable_income_per_capita'], nb_class=10)

In [ ]:
breaks

In [ ]:
df_comm['cut_jenks'] = pd.cut(df_comm['taxable_income_per_capita'],
                        bins=breaks,labels=False,include_lowest=True)

In [ ]:
df_comm.head()

In [ ]:
df_comm.cut_jenks.unique()

In [ ]:
color_var='cut_jenks'

In [ ]:
map_viz(df_comm,communities,'GMDNR','GMDNR','cut_jenks')

### 3: Logarithm

In [ ]:
df_comm['log_taxable_income_per_capita'] = np.log10(df_comm['taxable_income_per_capita'])

In [ ]:
map_viz(df_comm,communities,'GMDNR','GMDNR','log_taxable_income_per_capita')

### Try for yourself using the full dataset
Load it from https://thomann-public.s3.eu-west-1.amazonaws.com/jst-mapviz/bfs_communities.csv